# Time-varying bid problems with [PowerSimulations.jl](https://github.com/NREL-SIIP/PowerSimulations.jl)

**Originally Contributed by**: Sourabh Dalvi

## Introduction

PowerSimulations.jl supports the construction of Operations problems in power system
with three part cost bids for each time step. MarketBidCost allows the user to pass a
time-series of variable cost for energy and ancillary services jointly.
This example shows how to build a Operations problem with MarketBidCost and how to add
the time-series data to the devices.

## Dependencies
### Modeling Packages

In [1]:
using SIIPExamples
using PowerSystems
using PowerSimulations
const PSI = PowerSimulations

PowerSimulations

### Data management packages

In [2]:
using PowerSystemCaseBuilder
using Dates
using DataFrames
using TimeSeries

### Optimization packages

In [3]:
using Cbc #solver

### Data
This data depends upon the [RTS-GMLC](https://github.com/gridmod/rts-gmlc) dataset that we
assembled in the [tabular data parsing example](https://nbviewer.jupyter.org/github/NREL-SIIP/SIIPExamples.jl/blob/master/notebook/3_PowerSimulations_examples/01_operations_problems.ipynb).

In [4]:
include(
    joinpath(SIIPExamples.TEST_DIR, SIIPExamples.PSY_EX_FOLDER, "04_parse_tabulardata.jl"),
)

[ Info: Parsing csv files in Reserves ...
[ Info: Parsing csv data in DAY_AHEAD_regional_Flex_Down.csv ...
[ Info: Parsing csv data in DAY_AHEAD_regional_Flex_Up.csv ...
[ Info: Parsing csv data in DAY_AHEAD_regional_Reg_Down.csv ...
[ Info: Parsing csv data in DAY_AHEAD_regional_Reg_Up.csv ...
[ Info: Parsing csv data in DAY_AHEAD_regional_Spin_Up_R1.csv ...
[ Info: Parsing csv data in DAY_AHEAD_regional_Spin_Up_R2.csv ...
[ Info: Parsing csv data in DAY_AHEAD_regional_Spin_Up_R3.csv ...
[ Info: Parsing csv data in REAL_TIME_regional_Reg_Down.csv ...
[ Info: Parsing csv data in REAL_TIME_regional_Reg_Up.csv ...
[ Info: Parsing csv data in REAL_TIME_regional_Spin_Up_R1.csv ...
[ Info: Parsing csv data in REAL_TIME_regional_Spin_Up_R2.csv ...
[ Info: Parsing csv data in REAL_TIME_regional_Spin_Up_R3.csv ...
[ Info: Successfully parsed Reserves
[ Info: Parsing csv data in branch.csv ...
[ Info: Successfully parsed branch.csv
[ Info: Parsing csv data in bus.csv ...
[ Info: Successfully pa

System
┌───────────────────┬─────────────┐
│ Property          │ Value       │
├───────────────────┼─────────────┤
│ System Units Base │ DEVICE_BASE │
│ Base Power        │ 100.0       │
│ Base Frequency    │ 60.0        │
│ Num Components    │ 525         │
└───────────────────┴─────────────┘

Static Components
┌───────────────────────────────────────────┬───────┬────────────────────────┬──
│ Type                                      │ Count │ Has Static Time Series │ ⋯
├───────────────────────────────────────────┼───────┼────────────────────────┼──
│ Arc                                       │ 109   │ false                  │ ⋯
│ Area                                      │ 3     │ false                  │ ⋯
│ Bus                                       │ 73    │ false                  │ ⋯
│ GenericBattery                            │ 1     │ false                  │ ⋯
│ HVDCLine                                  │ 1     │ false                  │ ⋯
│ HydroDispatch                             │ 20    │ true                   │ ⋯
│ Line                                      │ 105   │ false                  │ ⋯
│ LoadZone                                  │ 3     │ true                   │ ⋯
│ PowerLoad                                 │ 51    │ true                   │ ⋯
│ RenewableDispatch                         │ 30    │ true                   │ ⋯
│ RenewableFix                              │ 31    │ true                   │ ⋯
│ TapTransformer                            │ 15    │ false                  │ ⋯
│ ThermalMultiStart                         │ 76    │ false                  │ ⋯
│ VariableReserve{PowerSystems.ReserveDown} │ 2     │ true                   │ ⋯
│ VariableReserve{PowerSystems.ReserveUp}   │ 5     │ true                   │ ⋯
└───────────────────────────────────────────┴───────┴────────────────────────┴──
                                                                1 column omitted

Time Series Summary
┌──────────────────────────────────┬─────────────────────┐
│ Property                         │ Value               │
├──────────────────────────────────┼─────────────────────┤
│ Components with time series data │ 142                 │
│ Total StaticTimeSeries           │ 182                 │
│ Total Forecasts                  │ 182                 │
│ Resolution                       │ 60 minutes          │
│ First initial time               │ 2020-01-01T00:00:00 │
│ Last initial time                │ 2020-01-01T00:00:00 │
│ Horizon                          │ 24                  │
│ Interval                         │ 0 minutes           │
│ Forecast window count            │ 1                   │
└──────────────────────────────────┴─────────────────────┘

### Creating the Time Series data for Energy bid

In [5]:
MultiDay = collect(
    DateTime("2020-01-01T00:00:00"):Hour(1):(DateTime("2020-01-01T00:00:00") + Hour(8783)),
);

### Adding a MarketBidCost time series
Here we add the energy bid
time series to the system. The TimeSeriesData that holds the energy bid data can be of any
type (i.e. `SingleTimeSeries` or `Deterministic`), but it has to be consistent with the existing
data in the `sys`. So, we'll first remove the existing `DeterministicSingleTimeSeries`, then add
the bid time series as `SingleTimeSeries`, then re-transform all of the time series in `sys`.

In [6]:
remove_time_series!(sys, DeterministicSingleTimeSeries)

for gen in get_components(ThermalGen, sys)
    varcost = get_operation_cost(gen)
    data = TimeArray(MultiDay, repeat([get_cost(get_variable(varcost))], 8784))
    _time_series = SingleTimeSeries("variable_cost", data)
    add_time_series!(sys, gen, _time_series)
    #set_variable_cost!(sys, gen, _time_series)
end

### Transforming SingleTimeSeries into Deterministic

In [7]:
horizon = 24;
interval = Dates.Hour(24);
transform_single_time_series!(sys, horizon, interval)

┌ Warning: There is only one forecast window. Setting interval = 0 seconds
└ @ InfrastructureSystems ~/.julia/packages/InfrastructureSystems/Z74ym/src/component.jl:712


In the [OperationsProblem example](https://nbviewer.jupyter.org/github/NREL-SIIP/SIIPExamples.jl/blob/master/notebook/3_PowerSimulations_examples/01_operations_problems.ipynb)
we defined a unit-commitment problem with a copper plate representation of the network.
Here, we want do define unit-commitment problem  with ThermalMultiStartUnitCommitment
formulation for thermal device representation.

For now, let's just choose a standard UC formulation.

In [8]:
uc_template = template_unit_commitment(network = CopperPlatePowerModel)


Operations Problem Specification
Transmission: PowerSimulations.CopperPlatePowerModel
Devices Models: 

	Type: PowerSystems.ThermalStandard
 	Formulation: PowerSimulations.ThermalBasicUnitCommitment

	Type: PowerSystems.HydroEnergyReservoir
 	Formulation: PowerSimulations.HydroDispatchRunOfRiver

	Type: PowerSystems.RenewableDispatch
 	Formulation: PowerSimulations.RenewableFullDispatch

	Type: PowerSystems.PowerLoad
 	Formulation: PowerSimulations.StaticPowerLoad

	Type: PowerSystems.InterruptibleLoad
 	Formulation: PowerSimulations.InterruptiblePowerLoad

	Type: PowerSystems.RenewableFix
 	Formulation: PowerSimulations.FixedOutput

	Type: PowerSystems.HydroDispatch
 	Formulation: PowerSimulations.HydroDispatchRunOfRiver

Branches Models: 

	Type: PowerSystems.Transformer2W
 	Formulation: PowerSimulations.StaticBranch

	Type: PowerSystems.Line
 	Formulation: PowerSimulations.StaticBranch

	Type: PowerSystems.HVDCLine
 	Formulation: PowerSimulations.HVDCDispatch

	Type: PowerSystems.T

And adjust the thermal generator formulation to use `ThermalMultiStartUnitCommitment`

In [9]:
set_device_model!(uc_template, ThermalMultiStart, ThermalMultiStartUnitCommitment)

Now we can build a 4-hour economic dispatch problem with the RTS data.

In [10]:
solver = optimizer_with_attributes(Cbc.Optimizer, "logLevel" => 1, "ratioGap" => 0.5)

problem = OperationsProblem(
    uc_template,
    sys,
    horizon = 4,
    optimizer = solver,
    balance_slack_variables = true,
)
build!(problem, output_dir = mktempdir())

PowerSimulations.BuildStatusModule.BuildStatus.BUILT = 0

And solve it ...

In [11]:
solve!(problem)

PowerSimulations.RunStatusModule.RunStatus.SUCCESSFUL = 0

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*